# Qualitative analysis of Compression

In [1]:
import pandas as pd
import pymove as pm
from pymove import MoveDataFrame
from pymove import filters
import pymove as pm
from pymove.visualization import folium as f, matplotlib as mpl
from pymove.utils import visual
from pymove.preprocessing import stay_point_detection, compression, segmentation
import numpy as np
from pyproj import Geod
from shapely.ops import nearest_points
import json
from shapely.geometry import Point, LineString, Polygon

from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from geopy.distance import geodesic 
from pyproj import Geod
from shapely.ops import nearest_points
import json

### Trajectory loading

In [2]:
df= pd.read_csv("datasets/cars_8.csv")
traj = MoveDataFrame(data=df, latitude='Latitude', longitude='Longitude', traj_id='id', datetime='Timestamp')
traj
#traj.plot_trajectory()

,id,datetime,lat,lon
0,8,2020-06-01 08:56:08.148000,50.861073,4.465373
1,8,2020-06-01 08:56:09.648000,50.861100,4.465386
2,8,2020-06-01 08:56:10.398000,50.861127,4.465398
3,8,2020-06-01 08:56:10.797745,50.861146,4.465406
4,8,2020-06-01 08:56:24.229685,50.861756,4.465684
...,...,...,...,...
4937,8,2020-06-04 21:56:59.514958,50.861842,4.465724
4938,8,2020-06-04 21:56:59.943340,50.861822,4.465716
4939,8,2020-06-04 21:57:01.415195,50.861756,4.465684
4940,8,2020-06-04 21:57:14.847135,50.861146,4.465406


### Compression

In [3]:
compressed = compression.compress_segment_stop_to_point(move_data=traj,dist_radius=20,time_radius=60*30)
compressed
#compressed.plot_trajectory()

,segment_stop,id,datetime,lat,lon,dist_to_prev,time_to_prev,speed_to_prev,stop,lat_mean,lon_mean
0,1,8,2020-06-01 08:56:08.148000,50.861073,4.465373,NaN,NaN,NaN,False,NaN,NaN
1,1,8,2020-06-01 08:56:09.648000,50.861100,4.465386,3.152482,1.500000,2.101655,False,NaN,NaN
2,1,8,2020-06-01 08:56:10.398000,50.861127,4.465398,3.152480,0.750000,4.203306,False,NaN,NaN
3,1,8,2020-06-01 08:56:10.797745,50.861146,4.465406,2.100317,0.399745,5.254144,False,NaN,NaN
4,2,8,2020-06-01 08:56:24.229685,50.861756,4.465684,NaN,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4937,368,8,2020-06-04 21:56:59.514958,50.861842,4.465724,1.174545,0.279437,4.203255,False,NaN,NaN
4938,368,8,2020-06-04 21:56:59.943340,50.861822,4.465716,2.250746,0.428382,5.254063,False,NaN,NaN
4939,368,8,2020-06-04 21:57:01.415195,50.861756,4.465684,7.733216,1.471855,5.254062,False,NaN,NaN
4940,369,8,2020-06-04 21:57:14.847135,50.861146,4.465406,NaN,NaN,NaN,False,NaN,NaN


In [4]:
series1 = traj[["datetime","lat","lon"]].values
series2 = compressed[["datetime","lat","lon"]].values
series1

array([[Timestamp('2020-06-01 08:56:08.148000'), 50.86107290000001,
        4.46537349999999],
       [Timestamp('2020-06-01 08:56:09.648000'), 50.861100166845816,
        4.465385801954695],
       [Timestamp('2020-06-01 08:56:10.398000'), 50.86112743367567,
        4.465398103909399],
       ...,
       [Timestamp('2020-06-04 21:57:01.415195'), 50.8617555,
        4.465684499999998],
       [Timestamp('2020-06-04 21:57:14.847135'), 50.86114559999999,
        4.465406299999998],
       [Timestamp('2020-06-04 21:57:16.446880'), 50.86107290000001,
        4.46537349999999]], dtype=object)

In [5]:
def dist(coord1, coord2):
    return geodesic(coord1, coord2).m

In [6]:
def compute_ED(initial, compressed):
    ED = []
    A = initial[0]
    compressed_path = LineString([Point(p[1], p[2]) for p in compressed]) 
    for i in range(len(initial)):
        B =  initial[i]
        point_B = Point(B[1], B[2])
        _point_B = compressed_path.interpolate(compressed_path.project(point_B))
        
        ED.append(dist((_point_B.x,_point_B.y), ((B[1], B[2]))))
    return ED
data = compute_ED(series1, series2)
sum(data)/len(data)

/home/pierre-cedric/miniconda3/envs/pymove/lib/python3.7/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


0.003769106543980067

In [7]:
traj.generate_speed_features()
compressed.generate_speed_features()

In [8]:
def compute_speed_score(initial,compressed):
    prec = None
    go = False
    hist_data = []
    error_col = []
    for index, row in compressed.iterrows():
        if go:
            time1 = pd.to_datetime(prec["datetime"])
            time2 = pd.to_datetime(row["datetime"])
            initial_ = initial[(initial["datetime"] >= time1) & (initial["datetime"] <= time2)]
            speed1 = prec["speed_to_prev"]
            speed2 = row["speed_to_prev"]
            speed_compressed_avg = (speed1+speed2)/2
            speed_error = (np.mean(initial_["speed_to_prev"]) - speed_compressed_avg)
            hist_data.append(speed_error)
            for _ in range(len(initial_)):
                error_col.append(speed_error)
            print()
            print(len(initial_),np.mean(initial_["speed_to_prev"]), speed_compressed_avg)
        prec = row
        go = True
    return hist_data, error_col 
data, error_col = compute_speed_score(traj, compressed)


2 2.1016547515364485 nan

2 3.152480539440271 3.152480539440271

2 4.72872537457163 4.72872537457163

2 5.254120301969257 5.254120301969257

2 5.254078433174728 5.254078433174728

2 5.254061708531983 5.254061708531983

2 2.6270313664269302 2.6270313664269302

2 1.050811831648915 1.050811831648915

2 3.152431837798912 3.152431837798912

2 4.728651589146331 4.728651589146331

2 6.304868922511563 6.304868922511563

2 8.05621227472562 8.05621227472562

2 8.75674133912494 8.75674133912494

2 4.378366403914548 4.378366403914548

2 1.050807587514208 1.050807587514208

2 3.1524269305124886 3.1524269305124886

2 5.254039723851113 5.254039723851113

2 7.355646133862065 7.355646133862065

2 8.581583503739562 8.581583503739562

2 4.378357750730329 4.378357750730329

2 1.0508049256664924 1.0508049256664924

2 3.1524140681051094 3.1524140681051094

2 2.101609142438617 2.101609142438617

2 1.050805270187637 1.050805270187637

2 3.152413750685283 3.152413750685283

2 2.1016084804976463 2.101608480497

In [39]:
compressed["speed-error"] = [data[0]]+data
compressed.to_csv("csv_results/pymove-comp-results.csv", index=False)


In [10]:
data = np.array(data)
data = data[~np.isnan(data)]

In [11]:
np.mean(np.abs(data[1:]))

0.10103666647031703